In [1]:
# Load AIEEE Data

In [1]:
%cd ../../

/home/souvic/mounted/btp/vahini/Name2Demographics


In [2]:
!nvidia-smi

Thu Mar 24 12:11:30 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.86       Driver Version: 470.86       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:3B:00.0 Off |                    0 |
| N/A   32C    P0    28W / 250W |      2MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla P100-PCIE...  Off  | 00000000:D8:00.0 Off |                    0 |
| N/A   

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"    
import tensorflow as tf

In [4]:
import sys  
sys.path.insert(0, 'Models/ERData/PreProcessing/')

from er_preprocess import *

In [5]:
er_df = pd.read_csv("Models/Data/ERData/er_data_with_majority_label.csv")

Load Models

In [6]:
#  other models
AIEEE_Muril = "Models/AIEEEGenderData/MurilBERT/SavedModel/MurilBERT"
CBSE_Muril = "Models/CBSEData/MurilBERT/SavedModel/MurilBERT"
ER_Muril = "Models/ERData/MurilBERT/SavedModel/BERTMuril"

In [7]:
%%capture
!pip install transformers[sentencepiece]
import transformers

In [8]:
from transformers import BertModel, AutoModelForSequenceClassification
from transformers import AutoTokenizer

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


In [9]:
aieee_model = AutoModelForSequenceClassification.from_pretrained(AIEEE_Muril)

In [10]:
cbse_model = AutoModelForSequenceClassification.from_pretrained(CBSE_Muril)

In [11]:
er_model = AutoModelForSequenceClassification.from_pretrained(ER_Muril)

In [12]:
tokenizer = AutoTokenizer.from_pretrained("google/muril-base-cased")

Predict Gender

In [13]:
from transformers import pipeline
from sklearn.metrics import classification_report


In [14]:
import math

def print_results(rmodel, tokenizer, df):
    pipe = pipeline(task="text-classification",model=rmodel, tokenizer=tokenizer)
    n = df.shape[0]
    predictions = []
    for i in range(math.ceil(n/1000)):
        df1 = df[i*1000:(i+1)*(1000)]
        onway = pipe(list(df1['Name']), batch_size=256)
        predictions = predictions + onway
#     import pdb; pdb.set_trace()
    labels = []
    for pred in predictions:
        if pred['label'] == 'LABEL_0':
            labels.append(0)
        else:
            labels.append(1)
    
    true = list(df['Gender'])
    report = classification_report(true, labels, digits = 4)
    print(classification_report(true, labels, digits = 4))
    return report

In [ ]:
report1 = print_results(aieee_model, tokenizer, er_df)

In [15]:
report2 = print_results(cbse_model, tokenizer, er_df)

RuntimeError: [enforce fail at CPUAllocator.cpp:67] . DefaultCPUAllocator: can't allocate memory: you tried to allocate 184320000 bytes. Error code 12 (Cannot allocate memory)